In [ ]:
# %load_ext autoreload
# %autoreload 1

In [1]:
import pickle as pkl
import pandas as pd
import tensorflow as tf
import tensorboard
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from feature_extraction import Dataset_allSeries
from model import model
import datetime

In [2]:
data_aux = pd.read_csv('../data/sird_constantes_dia.csv')

In [3]:
fechas = data_aux.fecha.unique()

In [ ]:
fechas

In [ ]:
"del 23 al 29 de Agosto son esas predicciones"

In [ ]:
# dates_to_traverse = fechas[174:]

In [4]:
def generate_seq(until, lenght, starts_with=0):
    aux = []
    for i in range(starts_with, starts_with + lenght):
        aux.append(i % until)
    return aux
def mae_imp(y_true, y_pred):
    return tf.reduce_sum(tf.math.reduce_mean(tf.math.abs(tf.math.subtract(y_true,y_pred)), axis=1),axis = -1)

In [ ]:
# for i in range(174, len(fechas) - 7, 7):
#     init_date = fechas[i]
#     end_date = fechas[i+7]
#     print(init_date,i , end_date)

In [ ]:
# init_date = fechas[len(fechas)-7]
# end_date = fechas[-1]
# print(init_date, end_date)

In [5]:
def r_squared(y_pred, y):
    import pdb; pdb.set_trace()
    SS_res = np.sum((y_pred - y)**2, axis=0)
    SS_tot = np.sum((y_pred - np.mean(y_pred,axis=(0,1)))**2, axis=0)
    coef = 1 - np.sum(SS_tot, axis = 0)/ np.sum(SS_res, axis = 0)
    return np.mean(coef)

In [6]:
# for i in range(len(fechas)-7, len(fechas) - 7,7):
init_date = fechas[len(fechas)-7]
end_date = fechas[-1]
print(init_date, len(fechas)-7, end_date)
train_set = data_aux[data_aux.fecha.isin(fechas[:len(fechas)-7])]
test_set =  data_aux[data_aux.fecha.isin(fechas[len(fechas)-7:len(fechas)-7 + 7])]
# test_set

2020-11-24 267 2020-11-30


In [7]:
# Biggest subset
data = {'train':train_set, 'test': test_set}
departamentos = data['train'].departamento.unique()
#departamentos = ['atlantico' , 'bogota', 'valle del cauca', 'antioquia']
window = 10
forecast_window = 7
num_dep = len(departamentos)

In [8]:
train_days = generate_seq(7, 164)
test_days = generate_seq(7, 9, (train_days[-1] + 1)%7)

In [9]:
from feature_extraction import Dataset_allSeries
dataset = Dataset_allSeries(window, forecast_window, departamentos)
train_dataset, val_dataset = dataset.get_data_set(data, train_days, test_days)
train_dataset

Cantidad de departamentos 33


<ZipDataset shapes: (((None, 33), (None, 33), (None,), (None,)), (None, 33)), types: ((tf.float64, tf.float64, tf.int32, tf.int32), tf.float64)>

In [10]:
dilation_rates=[1,2,2,4,4,6,6]
kernel_size=2
number_features=256
enc_convolution_params = []
for idx, dilation_rate in enumerate(dilation_rates):
    conv_params = dict()
    if idx == 0:
        conv_params["input_shape"] = (window, num_dep)
    conv_params["filters"] = number_features
    conv_params["kernel_size"] = kernel_size
    conv_params["padding"] = "causal"
    conv_params["dilation_rate"] = dilation_rate
    enc_convolution_params.append(conv_params)
print(enc_convolution_params)
print([50,num_dep*forecast_window])

[{'input_shape': (10, 33), 'filters': 256, 'kernel_size': 2, 'padding': 'causal', 'dilation_rate': 1}, {'filters': 256, 'kernel_size': 2, 'padding': 'causal', 'dilation_rate': 2}, {'filters': 256, 'kernel_size': 2, 'padding': 'causal', 'dilation_rate': 2}, {'filters': 256, 'kernel_size': 2, 'padding': 'causal', 'dilation_rate': 4}, {'filters': 256, 'kernel_size': 2, 'padding': 'causal', 'dilation_rate': 4}, {'filters': 256, 'kernel_size': 2, 'padding': 'causal', 'dilation_rate': 6}, {'filters': 256, 'kernel_size': 2, 'padding': 'causal', 'dilation_rate': 6}]
[50, 231]


In [11]:
net = model(enc_convolution_params, [50,num_dep*forecast_window])
net.compile(loss=mae_imp, optimizer="adam")

In [14]:
history = net.fit(train_dataset.batch(32),epochs=5)  #5

Epoch 1/5


TypeError: 'NoneType' object is not callable

In [ ]:
preds = net.predict(val_dataset.batch(1))
# y_pred = preds
# y = np.array([ y.numpy() for x, y in val_dataset ])
# SMAPE = np.sum(abs(y - y_pred)/((1/2)*(abs(y ) + abs(y_pred))),axis = 1)/7
# print(sum(SMAPE[0])/len(SMAPE[0]))
# print(sum(SMAPE[0][[1,3,4,-3]])/4)
# preds_train = net.predict(train_dataset.batch(1))
# y_train = np.array([ y.numpy() for x, y in train_dataset ])
# r = r_squared(preds_train, y_train)
# print('r_squared', r)
with open('preds_{}_{}.npy'.format(init_date,end_date), 'wb') as f:
    np.save(f, preds)

In [ ]:
#model = get_model()
net.compile(loss=mae_imp,
optimizer="adam")
"""es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', #val_loss
                                           patience=8,
                                           mode='min',
                                            restore_best_weights=True)"""
#logdir="logs/fit/"
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
history = net.fit(train_dataset.batch(32),
                epochs=5) 
                #callbacks = [es_callback,tensorboard_callback])

In [ ]:
preds = net.predict(val_dataset.batch(1))
y_pred = preds
y = np.array([ y.numpy() for x, y in val_dataset ])
SMAPE = np.sum(abs(y - y_pred)/((1/2)*(abs(y ) + abs(y_pred))),axis = 1)/7

In [ ]:
with open('preds_20_sep_27_sep.npy', 'wb') as f:
    np.save(f, preds)

In [ ]:
print(sum(SMAPE[0])/len(SMAPE[0]))
print(sum(SMAPE[0][[1,3,4,-3]])/4)

In [ ]:
SMAPE[0][[1,3,4,-3]]

In [ ]:
plt.plot(history.history['loss'],  label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
net.save_weights('model.h5')

In [ ]:
SMAPE

In [ ]:
sum(SMAPE[0])/len(SMAPE[0])

In [ ]:
departamentos

In [ ]:
SMAPE[0][[1,3,4,-3]]

In [ ]:
sum(MAPE[0][[1,3,4,-2]])/4

In [ ]:
resultados= pd.DataFrame({'departamento':departamentos, 'SMAPE':SMAPE[0]})

In [ ]:
resultados.to_csv('resultados_TCNN_exp_day_emb.csv')

In [ ]:
!pwd

In [ ]:
departamentos
capitales = ['atlantico' , 'bogota', 'valle del cauca', 'antioquia']
metric = sum([results[capital] for capital in capitales])/len(capitales)
metric

In [ ]:
+ np.finfo(np.float64).eps

In [ ]:
print(MAPE)

In [ ]:
departamentos

In [ ]:
#Calcular MAPE del dia anterior